In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random

In [2]:
df = pd.read_csv('../ML Assignment/DATA/Dataset_FE2.csv')
df['diagnosis']=df['diagnosis'].astype('category')

In [3]:
randlist=[1223, 91, 760, 1096, 670, 715, 373, 69, 538, 455]

In [4]:
train_array = [1]*10
test_array = [1]*10
X_train= [1]*10
X_test= [1]*10
y_train= [1]*10
y_test = [1]*10
for i in range(10):
    train_array[i] = df.sample(frac=0.67, random_state = randlist[i])
    test_array[i] = df.drop(train_array[i].index)
    trainData_Split = train_array[i].to_numpy()
    testData_Split = test_array[i].to_numpy()
    n_samples,n_features = trainData_Split.shape
    n_features -= 1
    X_train[i] = trainData_Split[:,1:]
    y_train[i] = trainData_Split[:,0]
    X_test[i] = testData_Split[:,1:]
    y_test[i] = testData_Split[:,0]

In [5]:
class LogisticRegression:
    
    def __init__(self, lr=0.01, n_iters=1000):
        self.lr = lr
        self.n_iters = n_iters
        self.weights = None
        self.bias = None
    
    def BGDFit(self, X, y):
        cost_history = []
        iterations = []
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        for i in range(self.n_iters):
            linear = np.dot(X, self.weights) + self.bias
            y_predict = self._sigmoid(linear)
            
            dw = (1/n_samples) * np.dot(X.T, (y_predict - y))
            db = (1/n_samples) * np.sum(y_predict - y)
            
            self.weights -= self.lr*dw
            self.bias -= self.lr*db
            
            cost=self.cost_function(X, y, self.weights, y_predict)
            cost_history.append(cost)
            iterations.append(i)
        
        y_coord = np.array(cost_history)
        x_coord = np.array(iterations)
        
        return x_coord, y_coord
            
            
    
    def MBGDFit(self, X, y, batch_size):
        cost_history = []
        iterations = []        
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
         
        for i in range(self.n_iters):

            idx = np.random.randint(n_samples-batch_size)
            X1 = X[idx:idx+batch_size]
            y1 = y[idx:idx+batch_size]
            linear = np.dot(X1, self.weights) + self.bias
            y_predict = self._sigmoid(linear)
            
            dw = (1/batch_size) * np.dot(X1.T, (y_predict - y1))
            db = (1/batch_size) * np.sum(y_predict - y1)
            
            self.weights -= self.lr*dw
            self.bias -= self.lr*db
            cost = self.cost_function(X1, y1, self.weights, y_predict)
            cost_history.append(cost)
            iterations.append(i)
        
        y_coord = np.array(cost_history)
        x_coord = np.array(iterations)
        
        return x_coord, y_coord
            
    def SGDFit(self, X, y):
        cost_history = []
        iterations = []        
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
         
        for i in range(self.n_iters):

            idx = np.random.randint(n_samples-1)
            X1 = X[idx:idx+1]
            y1 = y[idx:idx+1]
            linear = np.dot(X1, self.weights) + self.bias
            y_predict = self._sigmoid(linear)
            
            dw = np.dot(X1.T, (y_predict - y1))
            db = np.sum(y_predict - y1)
            
            self.weights -= self.lr*dw
            self.bias -= self.lr*db
            cost = self.cost_function(X1, y1, self.weights, y_predict)
            cost_history.append(cost)
            iterations.append(i)
        
        y_coord = np.array(cost_history)
        x_coord = np.array(iterations)
        
        return x_coord, y_coord
    
    def predict(self, X, threshold=0.5):
        linear = np.dot(X, self.weights) + self.bias
        y_sigmoid = self._sigmoid(linear)
        y_predict = [1 if i > threshold else 0 for i in y_sigmoid]
        return np.array(y_predict)
    
    def _sigmoid(self, x):
        return 1/(1+np.exp(-x)) 
    
    def cost_function(self, X, y, w, y_pred):
        n_samples = len(y)

        class1_cost = -y*np.log(y_pred)

        class2_cost = (1-y)*np.log(1-y_pred)

        cost = class1_cost - class2_cost 

        #Take the average cost
        cost = (cost.sum() / n_samples)
        costbias = 10e-8
        cost+=costbias

        return cost

In [6]:
def computeMetrics(y_actual, y_predicted):
    tp = np.sum((y_actual == 1) & (y_predicted == 1))
    tn = np.sum((y_actual == 0) & (y_predicted == 0))
    fp = np.sum((y_actual == 0) & (y_predicted == 1))        
    fn = np.sum((y_actual == 1) & (y_predicted == 0)) 
    accuracy = 100*(np.sum(y_actual == y_predicted)/len(y_actual))
    precision = tp/np.sum(tp+fp)
    recall = tp/np.sum(tp+fn)
    return accuracy, precision, recall

In [7]:
L = np.ones((10,3)).astype('object')

In [8]:
n=15000
warnings.filterwarnings('ignore')

In [9]:
fig,axes=plt.subplots(nrows=10, ncols=3, figsize=(100,100))
model_list=["Batch Gradient Descent", "Mini-Batch Gradient Descent", "Stochastic Gradient Descent"]
for i in range(10):
    x = [1]*3
    y = [1]*3    
    
    L[i][0] = LogisticRegression(lr=0.01, n_iters=n)
    L[i][1] = LogisticRegression(lr=0.001, n_iters=n)
    L[i][2] = LogisticRegression(lr=0.0001, n_iters=n)
    
    x[0],y[0]=L[i][0].BGDFit(X_train[i], y_train[i])
    x[1],y[1]=L[i][1].MBGDFit(X_train[i], y_train[i], batch_size=35)
    x[2],y[2]=L[i][2].SGDFit(X_train[i], y_train[i])
    
    for j in range(3):
        axes[i][j].set_title(model_list[j], fontsize = 28)
        axes[i][j].plot(x[j],y[j])
        axes[i][j].set_xlabel('Number of Iterations', fontsize = 28)
        axes[i][j].set_ylabel('Cost', fontsize = 28)


In [10]:
threshold_list = [0.3,0.4,0.5,0.6,0.7]

acc = np.ones((5,10,3)).astype('float')
prec = np.ones((5,10,3)).astype('float')
rec = np.ones((5,10,3)).astype('float')
for i in range(10):
    for j in range(5):
        for k in range(3):
            acc[j][i][k], prec[j][i][k], rec[j][i][k] = computeMetrics(y_test[i],L[i][k].predict(X_test[i],threshold=threshold_list[j]))

for j in range(5):
    for k in range(3):
        avgacc = np.mean(acc[j][:][k])
        print("Average accuracy of {} model with threshold value {} = {}".format(model_list[k], threshold_list[j], avgacc))
    print("\n")

Average accuracy of Batch Gradient Descent model with threshold value 0.3 = 94.14893617021278
Average accuracy of Mini-Batch Gradient Descent model with threshold value 0.3 = 92.7304964539007
Average accuracy of Stochastic Gradient Descent model with threshold value 0.3 = 94.8581560283688


Average accuracy of Batch Gradient Descent model with threshold value 0.4 = 95.92198581560285
Average accuracy of Mini-Batch Gradient Descent model with threshold value 0.4 = 94.68085106382979
Average accuracy of Stochastic Gradient Descent model with threshold value 0.4 = 96.45390070921985


Average accuracy of Batch Gradient Descent model with threshold value 0.5 = 97.3404255319149
Average accuracy of Mini-Batch Gradient Descent model with threshold value 0.5 = 96.9858156028369
Average accuracy of Stochastic Gradient Descent model with threshold value 0.5 = 97.16312056737588


Average accuracy of Batch Gradient Descent model with threshold value 0.6 = 96.09929078014186
Average accuracy of Mini-Bat

In [11]:
for j in range(5):
    for k in range(3):
        varacc = np.var(acc[j][:][k])
        print("Variance in accuracy of {} model with threshold value {} = {}".format(model_list[k], threshold_list[j], varacc))
    print("\n")

Variance in accuracy of Batch Gradient Descent model with threshold value 0.3 = 14.146672702580355
Variance in accuracy of Mini-Batch Gradient Descent model with threshold value 0.3 = 18.35923746290425
Variance in accuracy of Stochastic Gradient Descent model with threshold value 0.3 = 8.3622554197475


Variance in accuracy of Batch Gradient Descent model with threshold value 0.4 = 4.023942457622876
Variance in accuracy of Mini-Batch Gradient Descent model with threshold value 0.4 = 9.05387052965141
Variance in accuracy of Stochastic Gradient Descent model with threshold value 0.4 = 1.7604748252099967


Variance in accuracy of Batch Gradient Descent model with threshold value 0.5 = 0.18862230270107105
Variance in accuracy of Mini-Batch Gradient Descent model with threshold value 0.5 = 0.8173633117046429
Variance in accuracy of Stochastic Gradient Descent model with threshold value 0.5 = 1.949097127911071


Variance in accuracy of Batch Gradient Descent model with threshold value 0.6 = 

In [12]:
for j in range(5):
    for k in range(3):
        avgPrec = np.mean(prec[j][:][k])
        print("Average precision of {} model with threshold value {} = {}".format(model_list[k], threshold_list[j], avgPrec))
    print("\n")

Average precision of Batch Gradient Descent model with threshold value 0.3 = 0.8981532217148654
Average precision of Mini-Batch Gradient Descent model with threshold value 0.3 = 0.8689948535286582
Average precision of Stochastic Gradient Descent model with threshold value 0.3 = 0.8989471433915878


Average precision of Batch Gradient Descent model with threshold value 0.4 = 0.9325117800727556
Average precision of Mini-Batch Gradient Descent model with threshold value 0.4 = 0.9060775892171241
Average precision of Stochastic Gradient Descent model with threshold value 0.4 = 0.9297713622862318


Average precision of Batch Gradient Descent model with threshold value 0.5 = 0.9811535881958418
Average precision of Mini-Batch Gradient Descent model with threshold value 0.5 = 0.9550352799055543
Average precision of Stochastic Gradient Descent model with threshold value 0.5 = 0.9604093567251462


Average precision of Batch Gradient Descent model with threshold value 0.6 = 0.985216616290785
Avera

In [13]:
for j in range(5):
    for k in range(3):
        varPrec = np.var(prec[j][:][k])
        print("Variance in precision of {} model with threshold value {} = {}".format(model_list[k], threshold_list[j], varPrec))
    print("\n")

Variance in precision of Batch Gradient Descent model with threshold value 0.3 = 0.005994306382165961
Variance in precision of Mini-Batch Gradient Descent model with threshold value 0.3 = 0.006980999111163615
Variance in precision of Stochastic Gradient Descent model with threshold value 0.3 = 0.003318745021961788


Variance in precision of Batch Gradient Descent model with threshold value 0.4 = 0.0022488006022551867
Variance in precision of Mini-Batch Gradient Descent model with threshold value 0.4 = 0.004268999539300263
Variance in precision of Stochastic Gradient Descent model with threshold value 0.4 = 0.0008284736902918662


Variance in precision of Batch Gradient Descent model with threshold value 0.5 = 4.34622040312521e-05
Variance in precision of Mini-Batch Gradient Descent model with threshold value 0.5 = 0.0005774357046195344
Variance in precision of Stochastic Gradient Descent model with threshold value 0.5 = 0.0003432098765432103


Variance in precision of Batch Gradient De

In [14]:
for j in range(5):
    for k in range(3):
        avgRec = np.mean(rec[j][:][k])
        print("Average recall of {} model with threshold value {} = {}".format(model_list[k], threshold_list[j], avgRec))
    print("\n")

Average recall of Batch Gradient Descent model with threshold value 0.3 = 0.9680365296803654
Average recall of Mini-Batch Gradient Descent model with threshold value 0.3 = 0.9675925925925926
Average recall of Stochastic Gradient Descent model with threshold value 0.3 = 0.9866666666666667


Average recall of Batch Gradient Descent model with threshold value 0.4 = 0.9680365296803654
Average recall of Mini-Batch Gradient Descent model with threshold value 0.4 = 0.9675925925925926
Average recall of Stochastic Gradient Descent model with threshold value 0.4 = 0.9866666666666667


Average recall of Batch Gradient Descent model with threshold value 0.5 = 0.9497716894977168
Average recall of Mini-Batch Gradient Descent model with threshold value 0.5 = 0.9675925925925926
Average recall of Stochastic Gradient Descent model with threshold value 0.5 = 0.9688888888888889


Average recall of Batch Gradient Descent model with threshold value 0.6 = 0.91324200913242
Average recall of Mini-Batch Gradien

In [15]:
for j in range(5):
    for k in range(3):
        varRec = np.var(rec[j][:][k])
        print("Variance in recall of {} model with threshold value {} = {}".format(model_list[k], threshold_list[j], varRec))
    print("\n")

Variance in recall of Batch Gradient Descent model with threshold value 0.3 = 4.17005483622113e-05
Variance in recall of Mini-Batch Gradient Descent model with threshold value 0.3 = 4.286694101508886e-05
Variance in recall of Stochastic Gradient Descent model with threshold value 0.3 = 0.0


Variance in recall of Batch Gradient Descent model with threshold value 0.4 = 4.17005483622113e-05
Variance in recall of Mini-Batch Gradient Descent model with threshold value 0.4 = 4.286694101508886e-05
Variance in recall of Stochastic Gradient Descent model with threshold value 0.4 = 0.0


Variance in recall of Batch Gradient Descent model with threshold value 0.5 = 4.1700548362210624e-05
Variance in recall of Mini-Batch Gradient Descent model with threshold value 0.5 = 4.286694101508886e-05
Variance in recall of Stochastic Gradient Descent model with threshold value 0.5 = 0.0002765432098765438


Variance in recall of Batch Gradient Descent model with threshold value 0.6 = 0.000291903838535476
Va